---
> ## ✅ THIS NOTEBOOK WORKS WITH AZURE AI AGENTS
>
> **This notebook demonstrates conversation serialization with Azure AI agents using service-managed threads.**
>
> ### How This Works with Azure AI Agents
>
> Azure AI agents (`AzureAIAgentClient`) use **service-managed threads** where conversation history is automatically stored in Azure's cloud infrastructure. This notebook shows:
> - ✅ Creating conversations with Azure AI agents
> - ✅ Serializing thread state (lightweight ~50 bytes)
> - ✅ Deserializing and resuming conversations
> - ✅ Cloud-based message persistence
>
> ### Key Benefits
>
> - **Cloud Storage**: Azure automatically persists all messages
> - **Small Payload**: Only thread ID serialized (~50 bytes)
> - **Multi-Device**: Resume conversations anywhere
> - **Production Ready**: Enterprise-grade reliability
>
> ---


# Thread Serialization with Azure AI Agents

This notebook demonstrates how to use **service-managed threads** with Azure AI agents for persistent multi-turn conversations. 

## ✅ Works with Azure AI Agents

This notebook uses the **correct pattern** for Azure AI Foundry projects with `AzureAIAgentClient`.

## What You'll Learn

- ✅ How to create **service-managed threads** with Azure AI agents
- ✅ How to **serialize thread state** (just the thread ID)
- ✅ How to **deserialize and resume** conversations
- ✅ Understanding **cloud-based message persistence**
- ✅ **Lightweight serialization** for production applications

## Key Concepts

### Service-Managed Threads

Azure AI agents automatically manage conversation history in the cloud:
- **Automatic Persistence**: All messages stored in Azure
- **Thread ID**: Only identifier needed to resume conversations
- **Small Payload**: ~50 bytes vs 5000+ bytes for custom stores
- **Multi-Device**: Access conversations from anywhere

### Use Cases

- ? **Chat Applications**: Multi-session conversations
- ? **App Restarts**: Resume after deployments
- ? **Multi-Device**: Continue conversations on any device
- 🏢 **Enterprise**: Production-ready with automatic backup

> **Documentation**: [Multi-Turn Conversation Guide](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)


## Prerequisites

Before running this notebook:

1. **Azure CLI** authenticated: `az login --use-device-code`
2. **Environment variables** configured in `../.env`:
   - `AZURE_AI_PROJECT_ENDPOINT`
3. **Python packages** installed:
   ```bash
   pip install agent-framework agent-framework-azure-ai python-dotenv azure-identity
   ```


## Import Required Libraries

We'll import the necessary components for creating Azure AI agents and managing threads:


In [1]:
# Copyright (c) Microsoft. All rights reserved.

import json
import os

from dotenv import load_dotenv

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential


## Environment Configuration

Load environment variables from the parent directory's `.env` file:

In [2]:
# Load environment variables from parent directory
load_dotenv('../.env')

# Validate required environment variables
required_vars = ['AZURE_AI_PROJECT_ENDPOINT']
missing = [var for var in required_vars if not os.getenv(var)]

if missing:
    raise RuntimeError(f'Missing required environment variables: {missing}')

endpoint = os.getenv('AZURE_AI_PROJECT_ENDPOINT')
print(f'✅ AZURE_AI_PROJECT_ENDPOINT: {endpoint}')

✅ AZURE_AI_PROJECT_ENDPOINT: https://kd-foundry-project-resource.services.ai.azure.com/api/projects/kd-foundry-project


## Demo: Service-Managed Thread Serialization

This demonstration shows how to use service-managed threads with Azure AI agents:

1. **Create Agent**: Initialize an Azure AI agent (automatically uses service-managed threads)
2. **Start Conversation**: Begin a new thread with automatic cloud persistence
3. **Serialize State**: Save just the thread ID (~50 bytes)
4. **Deserialize State**: Restore the conversation from thread ID
5. **Resume Conversation**: Continue with full context from Azure cloud

### Real-World Applications

- **Multi-Session Apps**: Users can close and reopen conversations
- **Server Restarts**: Conversations persist across deployments
- **Multi-Device**: Access same conversation from web, mobile, desktop
- **Enterprise**: Automatic backup and disaster recovery


In [3]:
async def main():
    """
    Demonstrates service-managed thread serialization with Azure AI agents.
    
    This example shows:
    1. Creating an Azure AI agent (uses service-managed threads automatically)
    2. Having a conversation that's stored in Azure cloud
    3. Serializing the thread state (just the thread ID)
    4. Deserializing and resuming the conversation
    """
    print("=" * 70)
    print("Service-Managed Thread Demo with Azure AI Agents")
    print("=" * 70)
    print()

    # Create async context managers for Azure resources
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=endpoint, credential=credential) as project,
        AzureAIAgentClient(project_client=project) as client,
    ):
        # Create Azure AI agent
        # Service-managed threads are used automatically - no message_store needed
        agent = ChatAgent(
            chat_client=client,
            name="Joker",
            instructions="You are good at telling jokes.",
        )
        
        print(f"🤖 Created agent: {agent.display_name}")
        print(f"📋 Using: Service-managed threads (Azure cloud storage)")
        print()

        # ===== Phase 1: Start a new conversation =====
        print("─" * 70)
        print("Phase 1: Starting New Conversation")
        print("─" * 70)
        
        # Start a new thread - Azure automatically creates service-managed storage
        thread = agent.get_new_thread()
        print("✅ New service-managed thread created in Azure cloud")
        print()

        # Respond to user input
        query = "Tell me a joke about a pirate."
        print(f"👤 User: {query}")
        print(f"🤖 {agent.display_name}: ", end="")
        response = await agent.run(query, thread=thread)
        print(response.text)
        print()

        # ===== Phase 2: Serialize the thread state =====
        print("─" * 70)
        print("Phase 2: Serializing Thread State")
        print("─" * 70)
        
        # Serialize the thread state - only thread ID is saved
        serialized_thread = await thread.serialize()
        
        print("💾 Thread state serialized successfully")
        print(f"\n📄 Serialized data:\n{json.dumps(serialized_thread, indent=2)}\n")
        print("ℹ️  Notice:")
        print(f"   - Only thread ID stored: ~{len(json.dumps(serialized_thread))} bytes")
        print("   - All messages are in Azure cloud (not in serialized data)")
        print("   - Much smaller than custom stores (5000+ bytes)")
        print()
        print("💡 In production, you would:")
        print("   - Save this thread ID to your database")
        print("   - Associate it with user session/account")
        print("   - Use it to resume conversations later")
        print()

        # ===== Phase 3: Deserialize and resume conversation =====
        print("─" * 70)
        print("Phase 3: Resuming Conversation from Thread ID")
        print("─" * 70)
        
        # Simulate loading thread ID from storage
        print("📂 Loading thread ID from 'storage'...")
        
        # Deserialize the thread state - Azure retrieves messages from cloud
        resumed_thread = await agent.deserialize_thread(serialized_thread)
        print("✅ Thread state restored from Azure cloud")
        print()

        # Respond to user input - agent has full context from Azure
        query = "Now tell the same joke in the voice of a pirate, and add some emojis to the joke."
        print(f"👤 User: {query}")
        print(f"🤖 {agent.display_name}: ", end="")
        response = await agent.run(query, thread=resumed_thread)
        print(response.text)
        print()

        # ===== Summary =====
        print("=" * 70)
        print("✅ Demo Completed Successfully!")
        print("=" * 70)
        print()
        print("📊 Summary:")
        print(f"   - Azure AI agent maintained conversation context in cloud")
        print(f"   - Only thread ID serialized (~50 bytes)")
        print(f"   - All messages stored and retrieved from Azure automatically")
        print(f"   - Thread was saved and restored successfully")
        print()
        print("🔗 Learn More:")
        print("   https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python")

# Run the demo
await main()


Service-Managed Thread Demo with Azure AI Agents

🤖 Created agent: Joker
📋 Using: Service-managed threads (Azure cloud storage)

──────────────────────────────────────────────────────────────────────
Phase 1: Starting New Conversation
──────────────────────────────────────────────────────────────────────
✅ New service-managed thread created in Azure cloud

👤 User: Tell me a joke about a pirate.
🤖 Joker: 

[2025-10-25 20:57:07 - c:\Users\kapildhanger\OneDrive - Microsoft\Microsoft_Kapil\AzureCustomers\Everest\NYC_Workshop\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Why couldn't the pirate learn the alphabet?  

Because he always got lost at "C"!

──────────────────────────────────────────────────────────────────────
Phase 2: Serializing Thread State
──────────────────────────────────────────────────────────────────────
💾 Thread state serialized successfully

📄 Serialized data:
{
  "service_thread_id": "thread_YRE7wXhrSQOF1LLbOxY5VB9y",
  "chat_message_store_state": null
}

ℹ️  Notice:
   - Only thread ID stored: ~90 bytes
   - All messages are in Azure cloud (not in serialized data)
   - Much smaller than custom stores (5000+ bytes)

💡 In production, you would:
   - Save this thread ID to your database
   - Associate it with user session/account
   - Use it to resume conversations later

──────────────────────────────────────────────────────────────────────
Phase 3: Resuming Conversation from Thread ID
──────────────────────────────────────────────────────────────────────
📂 Loading thread ID from 'storage'...
✅ Thread state restored from Azure cl

## Next Steps

### Explore More Azure AI Features

1. **Multi-Agent**: Explore multi-agent workflows in the `agent-framework/workflows` folder
2. **Azure Functions**: See how to integrate agents with Azure Functions
3. **Agent Framework**: Check out other notebooks in `agent-framework/` for more examples


### Related Documentation

- 📖 [Multi-Turn Conversation Guide](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)
- 📖 [Agent Framework Overview](https://learn.microsoft.com/en-us/agent-framework/overview/agent-framework-overview/)
- 📖 [Azure AI Foundry Documentation](https://learn.microsoft.com/en-us/azure/ai-services/)
